ETL Objectives

The goal of the ETL process is to:

Clean and standardize raw support ticket data

Handle missing values and inconsistent formatting

Engineer new analytical fields required for BI and KPI calculation

Prepare a clean, analysis-ready dataset compatible with a dimensional model

Key observations

1/Text-heavy columns (subject, body, answer)

2/Multiple tag columns → need normalization

3/Missing values in tag_3 → tag_8

4/No timestamps → must be engineered

5/No AI indicators → must be engineered

🔹 Step 1: Load & Inspect Data

In [3]:
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/aa_dataset-tickets-multi-lang-5-2-50-version.csv")
df.info()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28587 entries, 0 to 28586
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   subject   24749 non-null  object
 1   body      28587 non-null  object
 2   answer    28580 non-null  object
 3   type      28587 non-null  object
 4   queue     28587 non-null  object
 5   priority  28587 non-null  object
 6   language  28587 non-null  object
 7   version   28587 non-null  int64 
 8   tag_1     28587 non-null  object
 9   tag_2     28574 non-null  object
 10  tag_3     28451 non-null  object
 11  tag_4     25529 non-null  object
 12  tag_5     14545 non-null  object
 13  tag_6     5874 non-null   object
 14  tag_7     2040 non-null   object
 15  tag_8     565 non-null    object
dtypes: int64(1), object(15)
memory usage: 3.5+ MB


,0
subject,3838
body,0
answer,7
type,0
queue,0
priority,0
language,0
version,0
tag_1,0
tag_2,13


🔹 Step 2: Standardize Column Names

In [4]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

🔹 Step 3: Handle Missing Values

In [5]:
tag_cols = [col for col in df.columns if col.startswith("tag_")]

df["all_tags"] = df[tag_cols].apply(
    lambda x: ", ".join(x.dropna()), axis=1
)

In [6]:
df.drop(columns=tag_cols, inplace=True)


🔹 Step 4: Text Cleaning (lightweight, BI-friendly)

In [7]:
def clean_text(text):
    if pd.isna(text):
        return ""
    return text.strip().lower()

for col in ["subject", "body", "answer"]:
    df[col] = df[col].apply(clean_text)


🔹 Step 5: Create Time Attributes

In [8]:
import numpy as np

df["created_date"] = pd.to_datetime("2024-01-01") + \
    pd.to_timedelta(np.random.randint(0, 180, size=len(df)), unit="D")

df["resolution_time_hours"] = np.where(
    df["priority"] == "high",
    np.random.randint(1, 12, size=len(df)),
    np.random.randint(4, 72, size=len(df))
)

df["resolved_date"] = df["created_date"] + \
    pd.to_timedelta(df["resolution_time_hours"], unit="h")


🔹 Step 6: Engineer AI Fields
AI-related attributes are engineered to simulate a realistic AI-assisted IT helpdesk environment for BI analysis.

In [9]:
df["ai_resolved"] = np.where(
    df["priority"].isin(["low", "medium"]),
    np.random.choice([1, 0], size=len(df), p=[0.6, 0.4]),
    0
)

df["ai_confidence_score"] = np.where(
    df["ai_resolved"] == 1,
    np.round(np.random.uniform(0.6, 0.95, size=len(df)), 2),
    None
)


🔹 Step 7: SLA Breach Flag

In [10]:
df["sla_breached"] = np.where(
    (df["priority"] == "high") & (df["resolution_time_hours"] > 24), 1,
    np.where(
        (df["priority"] == "medium") & (df["resolution_time_hours"] > 48), 1,
        0
    )
)


🔹 Step 8: Final Selection of BI Fields

In [11]:
final_columns = [
    "subject", "type", "queue", "priority", "language",
    "all_tags", "created_date", "resolved_date",
    "resolution_time_hours", "sla_breached",
    "ai_resolved", "ai_confidence_score"
]

df_clean = df[final_columns]


🔹 Step 9: Text Field Standardization & Final Data Cleaning

Empty strings and whitespace-only values in the subject field were standardized to null values and subsequently replaced with a descriptive placeholder. This ensures consistent textual representation while preserving analytical integrity.

In [12]:
import numpy as np

# Final standardization of empty text fields
df_clean.loc[:, "subject"] = (
    df_clean["subject"]
    .replace(r'^\s*$', np.nan, regex=True)
    .fillna("No Subject Provided")
)


🔹 Step 10: Export Clean Dataset

In [14]:
df_clean.to_csv("clean_it_helpdesk_tickets.csv", index=False)
